### This notebook is based on pytorch implementation of handyrl (https://github.com/DeNA/HandyRL) with minor changes

The inference part is taken from : https://www.kaggle.com/yuricat/smart-geese-trained-by-reinforcement-learning. As mentioned here https://www.kaggle.com/c/hungry-geese/discussion/218190, this model was trained for around 1,500 epochs with 800,000 games.

In [ ]:
import os
import sys
import yaml

# Clone handyrl (keep your own fork for flexibility)

In [ ]:
# !git clone https://github.com/kumar-shubham-ml/HandyRL.git
!git clone https://github.com/Dena/HandyRL.git
!cp -r HandyRL/. .

## Prepare Config

In [ ]:
%%writefile config.yaml

env_args:
    env: 'HungryGeese'
    source: 'handyrl.envs.kaggle.hungry_geese'


train_args:
    turn_based_training: False  # always False for Hungry Geese
    observation: False
    gamma: 0.8
    forward_steps: 32
    compress_steps: 4
    entropy_regularization: 2.0e-3
    entropy_regularization_decay: 0.3
    update_episodes: 500
    batch_size: 400  # GPU memory 10GB
    minimum_episodes: 100
    maximum_episodes: 5000  # RAM 64GB
    eval_rate: 0.1
    epochs: 120
    num_batchers: 7
    worker:
        num_parallel: 6
    lambda: 0.7
    policy_target: 'TD'
    value_target: 'TD'
    seed: 0
    restart_epoch: 0


worker_args:
    server_address: ''
    num_parallel: 8

In [ ]:
!ls

## Install Necessary Dependencies

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
# !pip3 install -r handyrl/envs/kaggle/requirements.txt

# Training

In [ ]:
from handyrl.train import train_main 
from handyrl.train import train_server_main
from handyrl.worker import worker_main
from handyrl.evaluation import eval_main
from handyrl.evaluation import eval_server_main
from handyrl.evaluation import eval_client_main

In [ ]:
with open('config.yaml') as f:
    args = yaml.safe_load(f)
print(args)

In [ ]:
train_main(args)

In [ ]:
!ls

In [ ]:
!ls models/

In [ ]:
! du -h

In [ ]:
!cp models/latest.pth /tmp/
!rm -r *
!cp /tmp/latest.pth .
!ls

# Preparing Submission File

In [ ]:
import pickle
import bz2
import base64
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

model_path = 'latest.pth'

weights = torch.load(model_path)

PARAM = base64.b64encode(bz2.compress(pickle.dumps(weights)))
state_dict = pickle.loads(bz2.decompress(base64.b64decode(PARAM)))

In [ ]:
type(state_dict)

In [ ]:

%%writefile submission.py

# This is a lightweight ML agent trained by self-play.
# After sharing this notebook,
# we will add Hungry Geese environment in our HandyRL library.
# https://github.com/DeNA/HandyRL
# We hope you enjoy reinforcement learning!


import pickle
import bz2
import base64
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


# Neural Network for Hungry Geese

class TorusConv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn):
        super().__init__()
        self.edge_size = (kernel_size[0] // 2, kernel_size[1] // 2)
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size=kernel_size)
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = torch.cat([x[:,:,:,-self.edge_size[1]:], x, x[:,:,:,:self.edge_size[1]]], dim=3)
        h = torch.cat([h[:,:,-self.edge_size[0]:], h, h[:,:,:self.edge_size[0]]], dim=2)
        h = self.conv(h)
        h = self.bn(h) if self.bn is not None else h
        return h


class GeeseNet(nn.Module):
    def __init__(self):
        super().__init__()
        layers, filters = 12, 32
        self.conv0 = TorusConv2d(17, filters, (3, 3), True)
        self.blocks = nn.ModuleList([TorusConv2d(filters, filters, (3, 3), True) for _ in range(layers)])
        self.head_p = nn.Linear(filters, 4, bias=False)
        self.head_v = nn.Linear(filters * 2, 1, bias=False)

    def forward(self, x):
        h = F.relu_(self.conv0(x))
        for block in self.blocks:
            h = F.relu_(h + block(h))
        h_head = (h * x[:,:1]).view(h.size(0), h.size(1), -1).sum(-1)
        h_avg = h.view(h.size(0), h.size(1), -1).mean(-1)
        p = self.head_p(h_head)
        v = torch.tanh(self.head_v(torch.cat([h_head, h_avg], 1)))

        return {'policy': p, 'value': v}


# Input for Neural Network

def make_input(obses):
    b = np.zeros((17, 7 * 11), dtype=np.float32)
    obs = obses[-1]

    for p, pos_list in enumerate(obs['geese']):
        # head position
        for pos in pos_list[:1]:
            b[0 + (p - obs['index']) % 4, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + (p - obs['index']) % 4, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + (p - obs['index']) % 4, pos] = 1
            
    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, pos_list in enumerate(obs_prev['geese']):
            for pos in pos_list[:1]:
                b[12 + (p - obs['index']) % 4, pos] = 1

    # food
    for pos in obs['food']:
        b[16, pos] = 1

    return b.reshape(-1, 7, 11)


# Load PyTorch Model

PARAM = b'XXXXXXXXXX'

state_dict = pickle.loads(bz2.decompress(base64.b64decode(PARAM)))
model = GeeseNet()
model.load_state_dict(state_dict)
model.eval()


# Main Function of Agent

obses = []

def agent(obs, _):
    obses.append(obs)
    x = make_input(obses)
    with torch.no_grad():
        xt = torch.from_numpy(x).unsqueeze(0)
        o = model(xt)
    p = o['policy'].squeeze(0).detach().numpy()

    actions = ['NORTH', 'SOUTH', 'WEST', 'EAST']
    return actions[np.argmax(p)]

In [ ]:
# Read in the submission file
with open('submission.py',) as file :
    filedata = file.read()

# Replace the target string
filedata = filedata.replace('XXXXXXXXXX', PARAM.decode("utf-8") )

# Write the file out again
with open('submission.py','w') as file:
    file.write(filedata)

In [ ]:
with open('submission.py',) as file :
    filedata = file.read()

In [ ]:
# print(filedata)

In [ ]:
from kaggle_environments import make
env = make("hungry_geese", debug=True)

env.reset()
env.run(['submission.py', 'submission.py', 'submission.py', 'submission.py'])
env.render(mode="ipython", width=800, height=700)